In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import deform_ops # library for my functions
from deform_ops import Deformer

# Multiresolution mesh editing: 
For this task, you will compute a mesh deformation based on the rotations and translations applied interactively to a subset of its vertices via the mouse. Let $H$ be the set of "handle" vertices that the user can manipulate (or leave fixed). We want to compute a deformation for the remaining vertices, denoted as $R$.

Let $\mathcal{S=(v,f)}$ be our input surface, represented as a triangle mesh. We want to compute a new surface that contains:
- the vertices in $H$ (the handles) translated/rotated using the user-provided transformation $t$, and
- the vertices in $R$ properly deformed using the algorithm described next.

The algorithm is divided in three phases:

1. removing high-frequency details:
`B = smooth_mesh(v,f,handle_indices,v)`
2. deforming the smooth mesh, and
`B' = smooth_mesh(v,f,handle_indices,v)`
3. transferring high-frequency details to the deformed surface.
`S' = B + details`

## Interactive Version

In [2]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
# v, f = igl.read_triangle_mesh('data/woody-lo.off')
# labels = np.load('data/woody-lo.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [3]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6)) # saves transformations in a martix
handle_indices = np.where(labels!=0)[0]
deformer_ = Deformer(v, f, handle_indices) 
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    # print('slices', slices)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [4]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [5]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    handle_indices = np.where(labels!=0)[0]
    # smooth original mesh (except vertex handles)
    B = deformer_.smooth_mesh(v)
    # smooth the target mesh (except vertex handles)
    B_prime =  deformer_.smooth_mesh(target_pos)
    # encode the details of B into a set of coefficients and tangents
    coeffs, tangents = deformer_.compute_detail_encoding(B)
    # apply the detail encoding to the target mesh
    B_prime_dets = deformer_.apply_detail_encoding(B_prime, coeffs, tangents)
    target_pos = B_prime + B_prime_dets
    return target_pos
pos_f.deformer = position_deformer

In [6]:
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

## Non-interactive version + Methodology 

#### Step 1: Smooth base surface (removal of high-frequency details) 

This is the `B = deformer.smooth_mesh(v)`

Note that `smooth_mesh`, takes in a value for the `new` positions of the vertices. Since smoothing the base surface does not have a new position of the vertices, we put in the original vertices as the new ones. 

We remove the high-frequency details from the vertices $R$ in $\mathcal{S}$ by minimizing the thin-plate energy, which involves solving a bi-Laplacian system arising from the quadratic energy minimization:
<img align="center" width="200" src="https://i.imgur.com/6IRzdBj.png">

where $o_H$ are the handle $H$'s vertex positions, $L_\omega$ is the cotan Laplacian of $\mathcal{S}$, and $M$ is the mass matrix of $\mathcal{S}$.

Notice that $L_\omega$ is the symmetric matrix consisting of the cotangent weights ONLY (without the division by Voronoi areas). In other words, it evaluates an "integrated" Laplacian rather than an "averaged" laplacian when applied to a vector of vertices. The inverse mass matrix appearing in the formula above then applies the appropriate rescaling so that the laplacian operator can be applied again (i.e., so that the Laplacian value computed at each vertex can be interpreted as a piecewise linear scalar field whose Laplacian can be computed).
This optimization will produce a mesh similar to the one in Figure 2. Note that the part of the surface that we want to deform is now free of high-frequency details. We call this mesh $\mathcal{B}$.

*Relevant `scipy` functions:* `scipy.sparse.csc_matrix`, `scipy.sparse.diags`, 

<!-- # Laplacian smoothing = doing explicit smoothung  -->
<!-- -- compute new pos of vertex as pos of vertex + displacement * laplacian  -->

#### Step 2: Deforming the smooth mesh

This is the `B_prime= deformer_.smooth_mesh(v_new)`

As before, `smooth_mesh`, takes in a value for the `new` positions of the vertices. Unlike the previous step, we now plug in `v_new` for the last parameter. 

The new deformed mesh is computed similarly to the previous step, by solving the minimization:
<img align="center" width="200" src="https://i.imgur.com/xv8ZcsA.png">
<!-- $$ 
\begin{aligned} \min_v& \quad v^T \textbf{L}_\omega \textbf{M}^{-1} \textbf{L}_\omega \textbf{v} \\
 \text{subject to}&
 \quad \textbf{v}_H = t(\textbf{o}_H),
\end{aligned}
$$ -->
where $t(\textbf{o}_H)$ are the new handle vertex positions after applying the user's transformation. We call this mesh $\mathcal{B}'$.

*Relevant `scipy` functions:* `scipy.sparse.linalg.spsolve` 

#### Step 3: Encode Displacement Vectors (in local frame) 

<!-- <img align="left" width="200" src="img/hand_bd.png"> 
<img align="left" width="200" src="img/hand_td.png">
<br clear="both"/>
<img align="left" width="200" src="img/woody_bd.png">
<img align="left" width="200" src="img/woody_td.png">
<br clear="both"/> -->

<!-- *Fig 4: Displacements on $B$ (left) and $B'$ (right)* -->

This is done in two steps :

* (1) `coeffs, tangent_vectors = compute_detail_encoding(B)` : which finds a reference frame for B by using:

  * $n_i$: the normal at $v$

  * $x_i$: normalized projection of one of $v$'s outgoing edges onto the tangent, and

  * $y_i$: the cross product of the normal and the tangent projection.

  These define a detail, $d$, as a set of coefficients \[d1,d2,d3\] multiplied by \[$x_i$, $y_i$, $n_i$\]. We save the coefficents to the `coeffs` array. We further save the values of all the $x_i$s to use in the computation of the reference frame for $B'$.

* (2) `dets = apply_detail_encoding(B_prime, coeffs, tangent_vectors)`: which returns the details we need to add back to reconstruct the deformed surface using:

  * $n_i'$: the normal at $v$ in `B_prime`

  * $x_i'$: the projection we obtained in step (1) which is saved in tangent_vectors

  * $y_i'$: the cross product of the normal of $B'$ and the tangent projection.

  We then compute the detail in $B'$ as each coefficient (saved in `coeffs`) multiplied by each component $x_i, y_i, n_i$. 

  These details are then added to $B'$ to give us our final surface $S'$. 

___ 
The high-frequency details on the original surface are extracted from $\mathcal{S}$ and transferred to $\mathcal{B}'$. We first encode the high-frequency details of $\mathcal{S}$ as displacements w.r.t. $\mathcal{B}$.
We define an orthogonal reference frame on every vertex $v$ of $\mathcal{B}$ using:
1. The unit vertex normal
2. The normalized projection of one of $v$'s outgoing edges onto the tangent plane defined by the vertex normal. A stable choice is the edge whose projection onto the tangent plane is longest.
3. The cross-product between (1) and (2)

For every vertex $v$, we compute the displacement vector that takes $v$ from $\mathcal{B}$ to $\mathcal{S}$ and represent it as a vector in $v$'s reference frame. 
For every vertex of $\mathcal{B}'$, we also construct a reference frame using the normal and the SAME outgoing edge we selected for $\mathcal{B}$ (not the longest in $\mathcal{B}'$; it is important that the edges used to build both reference frames are the same). We can now use the displacement vector components computed in the previous paragraph to define transferred displacement vectors in the new reference frames of $\mathcal{B}'$. See Figure 4 for an example.
Applying the transferred displacements to the vertices of $\mathcal{B}'$ generates the final deformed mesh $\mathcal{S}'$. See Figure 5 for an example.

<!-- <img align="left" width="200" src="img/hand_f.png">
<img align="left" width="200" src="img/woody_f.png">
<br clear="both"/> -->

*Fig 5: Final Deformation Results*

Recommended outputs:
- Provide screenshots for 4 different deformed meshes. For each example, provide a rendering of $\mathcal{S}$, $\mathcal{B}$, $\mathcal{B}'$ and $\mathcal{S}'$.


## Deforming 4 meshes:

In [8]:
mesh_files = ['data/hand.off']#, 'data/woody-lo.off', 'data/cactus.off', 'data/bumpy_plane.off']
labels_files = ['data/hand.label.npy']#, 'data/woody-lo.label.npy', 'data/cactus.label.npy', 'data/bumpy_plane.label.npy']
handle_choice = [2] #2,2,1]
shift_choice = [[0,0.5,0]]#, [0.5,0,0], [0,0.5,0], [0,0.5,0]]

In [9]:
for i in range(len(mesh_files)):
    # Loading in mesh data
    v, f = igl.read_triangle_mesh(mesh_files[i])
    labels = np.load(labels_files[i]).astype(int)
    v -= v.min(axis=0)
    v /= v.max()
    # obtaining handle vertex positions
    handle_idx = np.where(labels != 0)[0] 
    #  getting new positions (i.e. shifting handle1 by ... )
    new_pos = np.copy(v)
    handle1_idx = np.where(labels == handle_choice[i])[0] # indices of handle vertices
    for j in range(len(handle1_idx)):
        new_pos[handle1_idx[j]] = v[handle1_idx[j]]+np.array(shift_choice[i])
    deformer_ = Deformer(v, f, handle_idx)
    B = deformer_.smooth_mesh(v)
    B_prime = deformer_.smooth_mesh(new_pos)
    coeffs, tangent_vectors = deformer_.compute_detail_encoding(B)
    dets = deformer_. apply_detail_encoding(B_prime, coeffs, tangent_vectors)
    S = B_prime + dets 
    # Plotting the mesh
    # p1 = mp.subplot(v, f, c=labels, s=[1,4,0], shading={"wireframe": True})
    # p2 = mp.subplot(B, f, c=labels, data=p1, s=[1,4,1], shading={"wireframe": True})
    # p3 = mp.subplot(B_prime, f, c=labels, data=p1, s=[1,4,2], shading={"wireframe": True})
    # p4 = mp.subplot(S, f, c=labels, data=p1, s=[1,4,3], shading={"wireframe": True})
    p1 = mp.subplot(v, f, c=labels, s=[2,2,0], shading={"wireframe": True})
    p2 = mp.subplot(B, f, c=labels, data=p1, s=[2,2,1], shading={"wireframe": True})
    p3 = mp.subplot(B_prime, f, c=labels, data=p1, s=[2,2,2], shading={"wireframe": True})
    p4 = mp.subplot(S, f, c=labels, data=p1, s=[2,2,3], shading={"wireframe": True})
    

## Extra-credit: 
To achieve real-time performance, you must prefactor the sparse bi-Laplacian matrix appearing in both linear systems. After the user specifies vertex sets $H$ and $F$, you can factorize the matrix $L_\omega M^{-1} L_\omega$ (using a Cholesky $LL^T$ factorization) and then re-use the factorization to solve both linear systems efficiently. This is an optional part of the exercise; if your implementation does not achieve interactive frame-rates (10+ fps) on the gingerbread mesh, it will not receive the full score. This might require additional vectorizations.

*Available Packages*: `scikit-sparse`, `numba`.
*Relevant functions*: `sksparse.cholmod`, `numba.jit`, `numpy.einsum`, `scipy.sparse.linalg.splu`